In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

In [2]:
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
# from sklearn.metrics import root_mean_squared_error
import pandas as pd
from xgboost import XGBRegressor
import numpy as np

In [4]:
train_x  = pd.read_csv('/kaggle/input/thermaoracle-dataset/train_x.csv')
train_y = pd.read_csv('/kaggle/input/thermaoracle-dataset/train_y.csv', skip_blank_lines=False)


print(train_x.shape,train_y.shape)
train_y

(77158, 42) (77158, 1)


,Temperature (C)
0,4.583333
1,0.000000
2,9.955556
3,25.094444
4,3.816667
...,...
77153,7.183333
77154,14.983333
77155,13.577778
77156,-2.222222


In [5]:
train_x
train_df = pd.concat([train_x, train_y], axis=1)
df_clean = train_df.dropna()

train_x = df_clean[train_x.columns]
train_y = df_clean[train_y.columns]
print(train_x.shape,train_y.shape)
df_clean

(76443, 42) (76443, 1)


,Hour 0: Summary,Hour 0: Precip Type,Hour 0: Humidity,Hour 0: Wind Speed (km/h),Hour 0: Wind Bearing (degrees),Hour 0: Visibility (km),Hour 0: Pressure (millibars),Hour 1: Summary,Hour 1: Precip Type,Hour 1: Humidity,...,Hour 4: Visibility (km),Hour 4: Pressure (millibars),Hour 5: Summary,Hour 5: Precip Type,Hour 5: Humidity,Hour 5: Wind Speed (km/h),Hour 5: Wind Bearing (degrees),Hour 5: Visibility (km),Hour 5: Pressure (millibars),Temperature (C)
0,Partly Cloudy,rain,0.81,12.582479,12.291079,16.1000,1078.202030,Partly Cloudy,rain,0.81,...,15.1823,1066.629513,Mostly Cloudy,rain,0.73,16.084868,21.782768,16.1000,1062.069436,4.583333
1,Clear,snow,0.92,10.905645,91.415645,0.0000,1066.585351,Overcast,snow,0.92,...,11.2700,1084.652790,Overcast,snow,0.89,2.510763,69.780963,13.9587,1088.381365,0.000000
2,Windy and Overcast,rain,0.93,42.302599,317.376399,4.1699,1072.214882,Mostly Cloudy,rain,0.93,...,13.9748,1097.045649,Mostly Cloudy,rain,0.93,5.427351,143.309350,14.7315,1102.371539,9.955556
3,Partly Cloudy,rain,0.49,13.783278,360.749478,11.2056,1039.795328,Mostly Cloudy,rain,0.51,...,10.3523,1018.670757,Mostly Cloudy,rain,0.63,9.883718,12.470618,9.9820,1020.544830,25.094444
4,Mostly Cloudy,rain,0.89,14.073301,71.830201,11.1251,1095.292281,Overcast,rain,0.82,...,9.3380,1076.625917,Mostly Cloudy,rain,0.80,8.540998,100.353298,9.7083,1073.894083,3.816667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77153,Foggy,rain,1.00,-2.194588,262.660512,3.0590,1040.335999,Partly Cloudy,rain,0.95,...,15.0535,1029.456171,Partly Cloudy,rain,0.81,0.679895,285.137895,15.6492,1036.124820,7.183333
77154,Mostly Cloudy,rain,0.93,2.261096,132.350596,15.1823,1029.011961,Partly Cloudy,rain,0.90,...,14.9569,1030.364262,Overcast,rain,0.84,17.160861,83.939261,15.8263,1038.393351,14.983333
77155,Partly Cloudy,rain,0.89,8.420682,193.134582,10.8997,1113.770827,Partly Cloudy,rain,0.92,...,2.3506,1125.182342,Foggy,rain,0.93,9.010781,172.694281,2.9141,1123.664492,13.577778
77156,Mostly Cloudy,snow,0.96,3.749126,313.919126,11.2700,1079.075999,Mostly Cloudy,snow,0.99,...,2.0608,1103.636791,Overcast,snow,0.88,13.200040,313.710040,6.1180,1095.215124,-2.222222


In [6]:
import h2o
from h2o.automl import H2OAutoML

In [7]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.21" 2023-10-17; OpenJDK Runtime Environment (build 11.0.21+9-post-Ubuntu-0ubuntu120.04); OpenJDK 64-Bit Server VM (build 11.0.21+9-post-Ubuntu-0ubuntu120.04, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpevrq89zt
  JVM stdout: /tmp/tmpevrq89zt/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpevrq89zt/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,2 months and 24 days
H2O_cluster_name:,H2O_from_python_unknownUser_6qvqcx
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.250 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [8]:
df = h2o.H2OFrame(df_clean)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [9]:
df.describe(chunk_summary=True)

Rows:76443
Cols:43

Chunk compression summary: 
chunk_type    chunk_name                 count    count_percentage    size      size_percentage
------------  -------------------------  -------  ------------------  --------  -----------------
CXI           Sparse Integers            48       13.9535             105.1 KB  0.742397
C1N           1-Byte Integers (w/o NAs)  48       13.9535             451.1 KB  3.18554
C1S           1-Byte Fractions           48       13.9535             451.8 KB  3.19083
C4S           4-Byte Fractions           48       13.9535             1.8 MB    12.6799
C8D           64-bit Reals               152      44.186              11.1 MB   80.2013

Frame distribution summary: 
                 size     number_of_rows    number_of_chunks_per_column    number_of_chunks
---------------  -------  ----------------  -----------------------------  ------------------
127.0.0.1:54321  13.8 MB  76443             8                              344
mean             13.8 MB  76443             8                              344
min              13.8 MB  76443             8                              344
max              13.8 MB  76443             8                              344
stddev           0  B     0                 0                              0
total            13.8 MB  76443             8                              344

,Hour 0: Summary,Hour 0: Precip Type,Hour 0: Humidity,Hour 0: Wind Speed (km/h),Hour 0: Wind Bearing (degrees),Hour 0: Visibility (km),Hour 0: Pressure (millibars),Hour 1: Summary,Hour 1: Precip Type,Hour 1: Humidity,Hour 1: Wind Speed (km/h),Hour 1: Wind Bearing (degrees),Hour 1: Visibility (km),Hour 1: Pressure (millibars),Hour 2: Summary,Hour 2: Precip Type,Hour 2: Humidity,Hour 2: Wind Speed (km/h),Hour 2: Wind Bearing (degrees),Hour 2: Visibility (km),Hour 2: Pressure (millibars),Hour 3: Summary,Hour 3: Precip Type,Hour 3: Humidity,Hour 3: Wind Speed (km/h),Hour 3: Wind Bearing (degrees),Hour 3: Visibility (km),Hour 3: Pressure (millibars),Hour 4: Summary,Hour 4: Precip Type,Hour 4: Humidity,Hour 4: Wind Speed (km/h),Hour 4: Wind Bearing (degrees),Hour 4: Visibility (km),Hour 4: Pressure (millibars),Hour 5: Summary,Hour 5: Precip Type,Hour 5: Humidity,Hour 5: Wind Speed (km/h),Hour 5: Wind Bearing (degrees),Hour 5: Visibility (km),Hour 5: Pressure (millibars),Temperature (C)
type,enum,enum,real,real,real,real,real,enum,enum,real,real,real,real,real,enum,enum,real,real,real,real,real,enum,enum,real,real,real,real,real,enum,enum,real,real,real,real,real,enum,enum,real,real,real,real,real,real
mins,,,0.0,-4.998558178,-2.485756155,0.0,0.775924817,,,0.0,-4.998558178,-2.485756155,0.0,0.775924817,,,0.0,-4.998558178,-2.485756155,0.0,0.775924817,,,0.0,-4.998558178,-2.485756155,0.0,0.775924817,,,0.0,-4.998558178,-2.485756155,0.0,0.775924817,,,0.0,-4.998558178,-2.485756155,0.0,0.775924817,-21.82222222
mean,,,0.7382585717462676,7.68770459419747,188.07335364175526,10.402749952252009,1079.9523533979182,,,0.7382800256400192,7.687184700383662,188.07500555763465,10.402355471396989,1079.9585190443167,,,0.7382974242245853,7.686207042192047,188.08647606621284,10.402127586567785,1079.9640315096078,,,0.7383132530120475,7.685179877543599,188.0995780175572,10.401965834674193,1079.980525276885,,,0.7383246340410499,7.684213278740848,188.10856436632906,10.402156440746714,1079.9949913551293,,,0.7383320905772929,7.683372773408467,188.11835834027534,10.402426237850413,1080.0077972275064,11.91363187095693
maxs,,,1.0,57.76956497,367.96307,16.1,1888.288375,,,1.0,57.76956497,367.96307,16.1,1888.288375,,,1.0,57.76956497,367.96307,16.1,1888.288375,,,1.0,57.76956497,367.96307,16.1,1888.288375,,,1.0,57.76956497,367.96307,16.1,1888.288375,,,1.0,57.76956497,367.96307,16.1,1888.288375,38.86111111
sigma,,,0.19620101342424637,7.926179799086623,108.53178339922535,4.279720178411528,95.07815258113264,,,0.19619244798403626,7.926174448964252,108.52832734065272,4.280132650726961,95.08580144016148,,,0.19618651524323033,7.926194106297248,108.52402664825618,4.280439538778934,95.09109386022907,,,0.1961765462921923,7.92595406837375,108.51847894691683,4.280583259311112,95.03626884410158,,,0.1961629564811607,7.925446326450647,108.51552711999423,4.280650068071718,94.99067047554367,,,0.1961558942306607,7.925232490652715,108.51194003856294,4.2806691493938915,94.93406677303335,9.605736437708364
zeros,,,22,0,0,266,0,,,22,0,0,267,0,,,22,0,0,268,0,,,22,0,0,268,0,,,22,0,0,268,0,,,22,0,0,268,0,313
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,Partly Cloudy,rain,0.81,12.5824788,12.2910788,16.1,1078.20203,Partly Cloudy,rain,0.81,14.96170673,10.36800673,15.1823,1076.404755,Mostly Cloudy,rain,0.78,16.00149984,20.47399984,16.1,1073.641957,Mostly Cloudy,rain,0.76,20.51836004,12.65996004,16.1,1069.355043,Mostly Cloudy,rain,0.76,15.34198778,21.71608778,15.1823,1066.629513,Mostly Cloudy,rain,0.73,16.08486754,21.78276754,16.1,1062.069436,4.583333333
1,Clear,snow,0.92,10.90564455,91.41564455,0.0,1066.585351,Overcast,snow,0.92,9.217695643,91.33769564,7.084,1071.749572,Overcast,snow,0.92,8.975897217,96.27299722,5.6672,1075.98286,Overcast,snow,0.92,5.8033605,101.1433605,11.27,1080.726572,Overcast,snow,0.92,5.674115651,91.01411565,11.27,1084.65279,Overcast,snow,0.89,2.510762909,69.78096291,13.9587,1088.381365,0.0
2,Windy and Overcast,rain,0.93,42.3025

In [10]:
# train, test = df.split_frame(ratios=[0.8], seed = 1)
train =df

In [11]:
aml = H2OAutoML(max_models =25,
                balance_classes=True,
		seed =1)

In [12]:
aml.train(training_frame = train, y = 'Temperature (C)')

AutoML progress: |██████████████████████████████████████████████████████████████
05:42:34.275: DeepLearning_1_AutoML_1_20240316_50811 [DeepLearning def_1] failed: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for DeepLearning model: DeepLearning_1_AutoML_1_20240316_50811_cv_1.  Details: ERRR on field: _balance_classes: balance_classes requires classification.


█| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),12/42
# GBM base models (used / total),7/30
# XGBoost base models (used / total),5/9
# DRF base models (used / total),0/2
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [13]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,rmse,mse,mae,rmsle,mean_residual_deviance
StackedEnsemble_AllModels_1_AutoML_1_20240316_50811,3.92085,15.3731,2.98115,nan,15.3731
StackedEnsemble_BestOfFamily_1_AutoML_1_20240316_50811,4.01816,16.1456,3.07048,nan,16.1456
GBM_grid_1_AutoML_1_20240316_50811_model_4,4.0615,16.4958,3.10102,nan,16.4958
GBM_4_AutoML_1_20240316_50811,4.07089,16.5721,3.11351,nan,16.5721
GBM_1_AutoML_1_20240316_50811,4.07694,16.6215,3.12738,nan,16.6215
GBM_grid_1_AutoML_1_20240316_50811_model_14,4.09078,16.7345,3.12642,nan,16.7345
GBM_grid_1_AutoML_1_20240316_50811_model_9,4.11155,16.9048,3.11951,nan,16.9048
GBM_3_AutoML_1_20240316_50811,4.12824,17.0424,3.17205,nan,17.0424
GBM_grid_1_AutoML_1_20240316_50811_model_25,4.13217,17.0748,3.15435,nan,17.0748
GBM_2_AutoML_1_20240316_50811,4.17276,17.4119,3.21753,nan,17.4119


In [14]:
best_model = aml.get_best_model()
print(best_model)

Model Details
H2OStackedEnsembleEstimator : Stacked Ensemble
Model Key: StackedEnsemble_AllModels_1_AutoML_1_20240316_50811


Model Summary for Stacked Ensemble: 
key                                   value
------------------------------------  ----------------
Stacking strategy                     cross_validation
Number of base models (used / total)  12/42
# GBM base models (used / total)      7/30
# XGBoost base models (used / total)  5/9
# DRF base models (used / total)      0/2
# GLM base models (used / total)      0/1
Metalearner algorithm                 GLM
Metalearner fold assignment scheme    Random
Metalearner nfolds                    5
Metalearner fold_column
Custom metalearner hyperparameters    None

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 1.5554649493471748
RMSE: 1.2471828051040372
MAE: 0.9251243262664005
RMSLE: NaN
Mean Residual Deviance: 1.5554649493471748
R^2: 0.9831591296129871
Null degrees of freedom: 9925
Residual degrees of 

In [15]:
best_model.model_performance(test)

NameError: name 'test' is not defined

In [ ]:
model_path = h2o.save_model(model=best_model,path='/kaggle/working/model1', force=True)
print(model_path)

In [ ]:
explain_model = aml.explain(frame = test, figsize = (8,6))

In [ ]:
aml.explain_row(frame = test, row_index = 15, figsize = (8,6))